
#Dataset Preparation



#Importing Necessary Libraries

In [ ]:
import os
import random
import shutil
from pathlib import Path
import torch

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Downloading Dataset

In [ ]:
!kaggle datasets download -d iamtushara/face-detection-dataset
!unzip -q face-detection-dataset.zip -d .

#Cloning the YOLO *v5*

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5

In [ ]:
!pip install -r requirements.txt

#Selecting 1000 images and labels from original dataset for training

In [ ]:
train_images_path = '/content/merged/images/train/'
train_labels_path = '/content/merged/labels/train/'
val_images_path = '/content/merged/images/validation/'
val_labels_path = '/content/merged/labels/validation/'
new_dataset_folder = '/content/subset_dataset'

# Create a subset folder
try:
    os.makedirs(new_dataset_folder, exist_ok=True)
    os.makedirs(os.path.join(new_dataset_folder, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(new_dataset_folder, 'images', 'validation'), exist_ok=True)
    os.makedirs(os.path.join(new_dataset_folder, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(new_dataset_folder, 'labels', 'validation'), exist_ok=True)
    print("Subset folders created successfully.")
except OSError as e:
    print(f"Error: {e}")

# Verify folder creation
print("New dataset folder structure:")
for root, dirs, files in os.walk(new_dataset_folder):
    level = root.replace(new_dataset_folder, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)
    for file in files:
        print('{}{}'.format(subindent, file))

In [ ]:
subset_images_train_path = '/content/subset_dataset/images/train/'
subset_images_val_path = '/content/subset_dataset/images/validation/'
subset_labels_train_path = '/content/subset_dataset/labels/train/'
subset_labels_val_path = '/content/subset_dataset/labels/validation/'

# Function to count files in a directory
def count_files(directory, extensions):
    # List all files in the directory
    files = os.listdir(directory)
    # Count the number of files with specified extensions
    count = sum(1 for file in files if os.path.isfile(os.path.join(directory, file)) and file.lower().endswith(tuple(extensions)))
    return count

# Count image files in training and validation sets
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
num_images_train = count_files(subset_images_train_path, image_extensions)
num_images_val = count_files(subset_images_val_path, image_extensions)

# Count label files in training and validation sets
label_extensions = ['.txt']  # Add more extensions if needed
num_labels_train = count_files(subset_labels_train_path, label_extensions)
num_labels_val = count_files(subset_labels_val_path, label_extensions)

# Print counts
print("Number of image files in the training set:", num_images_train)
print("Number of image files in the validation set:", num_images_val)
print("Number of label files in the training set:", num_labels_train)
print("Number of label files in the validation set:", num_labels_val)


##Counting the no of images that we copied in subset folder for training

In [ ]:
subset_images_path = '/content/subset_dataset/train_val/labels/'

# List all files in the directory
files = os.listdir(subset_images_path)
# image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
image_extensions = ['.txt']  # Add more extensions if needed
image_count = sum(1 for file in files if os.path.isfile(os.path.join(subset_images_path, file)) and file.lower().endswith(tuple(image_extensions)))

print("Number of images:", image_count)


#Training the Model

In [ ]:
!python train.py --weights yolov5s.pt --data /content/dataset.yaml --epochs 20 --batch-size 16 --imgsz 640 --project /content/yolov5/runs/train --name exp_subset --exist-ok --seed 0 --workers 8

#Visualizing the results

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/yolov5/runs/train

#Testing Part

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
img = '/content/Shameer_unseen.png'

# Inference
results = model(img, augment=True)

# Results
results.print()

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
img = '/content/unseen_image.jpg'

# Inference
results = model(img, augment=True)

# Results
results.show()